In [28]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from criticalpath import Node
import plotly.graph_objects as go

In [3]:
x1a_duration=6
x1b_duration=6
x1c_duration=6
task = [('X1A',{'Duration':x1a_duration}),('X1B',{'Duration':x1b_duration}),('X1C',{'Duration':x1c_duration}),('X2',{'Duration':1}),('X3',{'Duration':1}),
        ('X4',{'Duration':1}),('X5',{'Duration':4}),('X6',{'Duration':6}),('X7',{'Duration':4}),('X8',{'Duration':1}),
        ('X9',{'Duration':2}),('X10',{'Duration':1}),('X11',{'Duration':1}),('X12',{'Duration':1}),('X13',{'Duration':1}),
        ('X14',{'Duration':5}),('X15',{'Duration':7}),('X16',{'Duration':5}),('X17',{'Duration':7}),('X18',{'Duration':10}),
        ('X19',{'Duration':7}),('X20',{'Duration':3}),('X21',{'Duration':5}),('X22',{'Duration':3}),('X23',{'Duration':6}),
        ('X24',{'Duration':8}),('X25',{'Duration':2}),('X26',{'Duration':4}),('X27',{'Duration':5}),('X28',{'Duration':7}),
        ]

# Define dependency
depend = [('X1A','X2'),('X1A','X5'),('X1B','X3'),('X1B','X6'),('X1C','X4'),('X1C','X7'),
          ('X2','X8'),('X5','X8'),('X3','X9'),('X6','X9'),('X4','X10'),('X7','X10'),('X8','X11'),('X8','X14'),('X9','X12'),
          ('X9','X15'),('X10','X13'),('X10','X16'),('X11','X17'),('X14','X17'),('X12','X18'),('X15','X18'),('X13','X19'),
          ('X16','X19'),('X17','X20'),('X18','X21'),('X19','X22'),('X17','X23'),('X18','X24'),('X23','X25'),('X24','X25'),
          ('X25','X26'),('X25','X27'),('X20','X28'),('X26','X28'),('X27','X28'),('X21','X28'),('X22','X28')]

In [4]:
pos_nodes = {"X1A": (51, 81),
             "X1B": (51, 54),
             "X1C": (51, 24),
             "X2": (87, 81),
             "X3": (87, 54), 
             "X4": (87, 27), 
             "X5": (87, 66),
             "X6": (87, 39),
             "X7": (87, 12),
             "X8": (114, 75),
             "X9": (114, 48),
             "X10": (114, 18),
             "X11": (141, 81),
             "X12": (141, 54),
             "X13": (141, 24),
             "X14": (141, 69),
             "X15": (141, 39), 
             "X16": (141, 12), 
             "X17": (171, 75), 
             "X18": (171, 48), 
             "X19": (171, 18),
             "X20": (204, 75),
             "X21": (204, 54),
             "X22": (204, 18),
             "X23": (204, 60),
             "X24": (204, 33),
             "X25": (231, 48),
             "X26": (261, 54),
             "X27": (261, 42),
             "X28": (288, 48)}

In [5]:
def forward_pass(df):
    ES = [0] * len(df)
    EF = [0] * len(df)

    for i, row in df.iterrows():
        if row['Predecessors'] == 'None':  # If no predecessors, start at time 0
            ES[i] = 0
        else:
            preds = row['Predecessors'].split(', ')  # Get all predecessors
            ES[i] = max([EF[df[df['Task'] == p].index[0]] for p in preds])  # Max EF of predecessors
        EF[i] = ES[i] + row['Duration']  # EF = ES + Duration

    df['ES'] = ES
    df['EF'] = EF
    return df

# Backward pass
def backward_pass(df):
    LS = [0] * len(df)
    LF = [0] * len(df)

    max_EF = df['EF'].max()
    for i in reversed(range(len(df))):
        row = df.iloc[i]
        # Check if the task has successors (tasks with this task as predecessor)
        successors = df[df['Predecessors'].str.contains(row['Task'], na=False)]
        if successors.empty:  # If no successors, set LF to max EF
            LF[i] = max_EF
        else:
            LF[i] = min([LS[df[df['Task'] == s].index[0]] for s in successors['Task']])
        LS[i] = LF[i] - row['Duration']

    df['LS'] = LS
    df['LF'] = LF
    return df

# Calculate Slack and Critical Path
def calculate_slack_and_critical(df):
    df['Slack'] = df['LS'] - df['ES']
    df['Critical'] = df['Slack'] == 0  # Critical if Slack is 0
    return df

In [6]:
# Create task dictionary
taskDict = {t[0]: t[1]['Duration'] for t in task}

# Create predecessor and successor dictionaries
predecessors = {t[0]: [] for t in task}
successors = {t[0]: [] for t in task}

# Populate predecessor and successor dictionaries
for dep in depend:
    predecessors[dep[1]].append(dep[0])
    successors[dep[0]].append(dep[1])

# Convert the data into a DataFrame for easy viewing
data = {
    'Task': list(taskDict.keys()),
    'Duration': list(taskDict.values()),
    'Predecessors': [', '.join(predecessors[task]) if predecessors[task] else 'None' for task in taskDict],
    'Successors': [', '.join(successors[task]) if successors[task] else 'None' for task in taskDict]
}

df = pd.DataFrame(data)



In [17]:
def check_critical_path_change_with_cost(task_to_modify, new_duration):
    global global_df  # Use the global variable to access the original DataFrame

    # Step 1: Create a local copy of the global DataFrame to perform operations
    df = global_df.copy()

    # Step 2: Calculate the initial critical path
    original_df = df.copy()
    original_df = forward_pass(original_df)
    original_df = backward_pass(original_df)
    original_df = calculate_slack_and_critical(original_df)
    initial_critical_path = original_df[original_df['Critical'] == True]['Task'].tolist()

    # Step 3: Modify the specified task's duration (e.g., X5)
    original_duration = df.loc[df['Task'] == task_to_modify, 'Duration'].values[0]  # Store the original duration
    df.loc[df['Task'] == task_to_modify, 'Duration'] = new_duration  # Temporarily change the duration

    # Step 4: Recalculate the critical path after modifying the task's duration
    modified_df = forward_pass(df)
    modified_df = backward_pass(modified_df)
    modified_df = calculate_slack_and_critical(modified_df)
    modified_critical_path = modified_df[modified_df['Critical'] == True]['Task'].tolist()


    # Step 5: Compare the initial and modified critical paths
    critical_path_changed = initial_critical_path != modified_critical_path

    

    # Step 6: Revert the duration of the task to its original value
    global_df.loc[global_df['Task'] == task_to_modify, 'Duration'] = original_duration  # Restore original duration

    return initial_critical_path, modified_critical_path, critical_path_changed 


In [18]:
global_df = df.copy()

def iterative_x18_reduction():
    global global_df
    
    # Initialize the duration of X18 to 10 (as per the given problem)
    x18_duration = 10

    # Step 1: Set the durations for X20, X21, and X22
    x20_new_duration = 1.5
    x21_new_duration = 2.5
    x22_new_duration = 3

    # Step 2: Start iterating to reduce X18
    while x18_duration >= 5:
        # Modify X18 duration by reducing 1 unit iteratively
        global_df.loc[global_df['Task'] == 'X18', 'Duration'] = x18_duration

        # Recalculate project duration after reducing X18
        df = forward_pass(global_df.copy())
        df = backward_pass(df)
        df = calculate_slack_and_critical(df)
        
        project_duration = df[df['Critical'] == True]['EF'].max()
        
        
        # Step 3: Check if project duration is still greater than 56
        if project_duration <= 56:
            break  # Exit loop if project duration is optimized to be less than or equal to 56 weeks
        
        # Step 4: If the project duration is still greater than 56
        if project_duration > 56:
            critical_tasks = df[df['Critical'] == True]['Task'].tolist()
            
            # Step 5: Check if X20, X21, or X22 are on the critical path
            if any(task in critical_tasks for task in ['X20', 'X21', 'X22']):
                print("Critical path contains X20, X21, or X22. Adjusting their durations...")

                # Modify the durations of X20, X21, and X22
                if 'X20' in critical_tasks:
                    global_df.loc[global_df['Task'] == 'X20', 'Duration'] = x20_new_duration
                    print(f"X20 duration set to {x20_new_duration} weeks.")
                if 'X21' in critical_tasks:
                    global_df.loc[global_df['Task'] == 'X21', 'Duration'] = x21_new_duration
                    print(f"X21 duration set to {x21_new_duration} weeks.")
                if 'X22' in critical_tasks:
                    global_df.loc[global_df['Task'] == 'X22', 'Duration'] = x22_new_duration
                    print(f"X22 duration set to {x22_new_duration} weeks.")
                
                # Restart the reduction process for X18 after adjusting X20, X21, X22
                print(f"Restarting X18 reduction process after adjusting X20, X21, and X22...")
                x18_duration = 10  # Reset X18 to the original value to restart reduction
            else:
                # Reduce X18 duration further
                x18_duration -= 1
    
    
        

# Run the iterative function
iterative_x18_reduction()


In [19]:
def optimize_project_duration_with_cost():
    global global_df

    # Step 1: Calculate the initial project duration based on the critical path
    df = global_df.copy()
    df = forward_pass(df)
    df = backward_pass(df)
    df = calculate_slack_and_critical(df)

    # Find the total project duration (Max EF of the critical path tasks)
    project_duration = df[df['Critical'] == True]['EF'].max()

    # Initialize task durations
    x27_duration = 2
    x5_duration = 2
    x6_duration = 3
    x7_duration = 2
    x18_duration = 5
    x20_duration = 1.5
    x21_duration = 2.5
    x22_duration = 1.5

    # Step 2: If project duration is greater than 56 weeks, try to reduce it by modifying tasks
    if project_duration > 56:
        
        # Modify task X27 first
        result_x27 = check_critical_path_change_with_cost('X27', new_duration=x27_duration)
        global_df.loc[global_df['Task'] == 'X27', 'Duration'] = x27_duration

        # Recalculate the project duration after modifying X27
        df = forward_pass(global_df.copy())
        df = backward_pass(df)
        df = calculate_slack_and_critical(df)
        project_duration = df[df['Critical'] == True]['EF'].max()

        # Step 3: If project duration is greater than 56 weeks and the critical path changes, modify other tasks
        if project_duration > 56:
            
            # Modify X5
            result_x5 = check_critical_path_change_with_cost('X5', new_duration=x5_duration)
            global_df.loc[global_df['Task'] == 'X5', 'Duration'] = x5_duration

            # Modify X6
            result_x6 = check_critical_path_change_with_cost('X6', new_duration=x6_duration)
            global_df.loc[global_df['Task'] == 'X6', 'Duration'] = x6_duration

            # Modify X7
            result_x7 = check_critical_path_change_with_cost('X7', new_duration=x7_duration)
            global_df.loc[global_df['Task'] == 'X7', 'Duration'] = x7_duration

            # Recalculate the project duration after modifying X5, X6, and X7
            df = forward_pass(global_df.copy())
            df = backward_pass(df)
            df = calculate_slack_and_critical(df)
            project_duration = df[df['Critical'] == True]['EF'].max()

            # Step 4: If project duration is still greater than 56 weeks, call iterative_x18_reduction()
            if project_duration > 56:
                iterative_x18_reduction()  # Further reduction of X18 to optimize project duration
        else:
            # Update the project duration with the adjustments if within the threshold
            global_df = df.copy()

            
            
# Run the optimization function
optimize_project_duration_with_cost()


In [20]:
def run_optimization_and_collect_results():
    results = []  # To store results for each combination

    # Iterate over the possible values of X1A, X1B, and X1C
    for x1a_duration in (6, 16, 17, 18, 19, 20, 21):
        for x1b_duration in (6, 16, 17, 18, 19, 20, 21):
            for x1c_duration in (6, 16, 17, 18, 19, 20, 21):
                
                # Step 1: Set X1A, X1B, and X1C durations
                global_df.loc[global_df['Task'] == 'X1A', 'Duration'] = x1a_duration
                global_df.loc[global_df['Task'] == 'X1B', 'Duration'] = x1b_duration
                global_df.loc[global_df['Task'] == 'X1C', 'Duration'] = x1c_duration

                # Step 2: Run the optimization function
                optimize_project_duration_with_cost()

                # Step 3: Collect the final values after optimization
                df = global_df.copy()  # Use the updated global dataframe
                df = forward_pass(df)
                df = backward_pass(df)
                df = calculate_slack_and_critical(df)

                # Get the critical path and final project duration
                critical_path = df[df['Critical'] == True]['Task'].tolist()
                project_duration = df[df['Critical'] == True]['EF'].max()

                # Get the relevant task durations after optimization
                x1a_final = df.loc[df['Task'] == 'X1A', 'Duration'].values[0]
                x1b_final = df.loc[df['Task'] == 'X1B', 'Duration'].values[0]
                x1c_final = df.loc[df['Task'] == 'X1C', 'Duration'].values[0]
                x5_final = df.loc[df['Task'] == 'X5', 'Duration'].values[0]
                x6_final = df.loc[df['Task'] == 'X6', 'Duration'].values[0]
                x7_final = df.loc[df['Task'] == 'X7', 'Duration'].values[0]
                x18_final = df.loc[df['Task'] == 'X18', 'Duration'].values[0]
                x20_final = df.loc[df['Task'] == 'X20', 'Duration'].values[0]
                x21_final = df.loc[df['Task'] == 'X21', 'Duration'].values[0]
                x22_final = df.loc[df['Task'] == 'X22', 'Duration'].values[0]
                x27_final = df.loc[df['Task'] == 'X27', 'Duration'].values[0]

                # Append the result for this combination
                results.append({
                    'X1A': x1a_final,
                    'X1B': x1b_final,
                    'X1C': x1c_final,
                    'X5': x5_final,
                    'X6': x6_final,
                    'X7': x7_final,
                    'X18': x18_final,
                    'X20': x20_final,
                    'X21': x21_final,
                    'X22': x22_final,
                    'X27': x27_final,
                    'Project Duration': project_duration,
                    'Critical Path': critical_path
                })

    # Convert the results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Run the optimization and collect results into a DataFrame
optimization_results_df = run_optimization_and_collect_results()

# Display the resulting DataFrame
optimization_results_df

,X1A,X1B,X1C,X5,X6,X7,X18,X20,X21,X22,X27,Project Duration,Critical Path
0,6,6,6,4,6,4,10,3,5,3,5,53,"[X1B, X6, X9, X15, X18, X24, X25, X27, X28]"
1,6,6,16,4,6,4,10,3,5,3,5,53,"[X1B, X6, X9, X15, X18, X24, X25, X27, X28]"
2,6,6,17,4,6,4,10,3,5,3,5,53,"[X1B, X6, X9, X15, X18, X24, X25, X27, X28]"
3,6,6,18,4,6,4,10,3,5,3,5,53,"[X1B, X6, X9, X15, X18, X24, X25, X27, X28]"
4,6,6,19,4,6,4,10,3,5,3,5,53,"[X1B, X6, X9, X15, X18, X24, X25, X27, X28]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,21,21,17,2,3,2,5,3,5,3,2,59,"[X1B, X6, X9, X15, X18, X24, X25, X26, X28]"
339,21,21,18,2,3,2,5,3,5,3,2,59,"[X1B, X6, X9, X15, X18, X24, X25, X26, X28]"
340,21,21,19,2,3,2,5,3,5,3,2,59,"[X1B, X6, X9, X15, X18, X24, X25, X26, X28]"
341,21,21,20,2,3,2,5,3,5,3,2,59,"[X1B, X6, X9, X15, X18, X24, X25, X26, X28]"


In [32]:
x1a_duration=int(input("Please enter the value of x1a duration from 6,16,17,18,19,20,21"))
x1b_duration=int(input("Please enter the value of x1b duration from 6,16,17,18,19,20,21"))
x1c_duration=int(input("Please enter the value of x1c duration from 6,16,17,18,19,20,21"))
matching_row = optimization_results_df[
    (optimization_results_df['X1A'] == x1a_duration) &
    (optimization_results_df['X1B'] == x1b_duration) &
    (optimization_results_df['X1C'] == x1c_duration)
]

x5_value = matching_row['X5'].values[0]
x6_value = matching_row['X6'].values[0]
x7_value = matching_row['X7'].values[0]
x18_value = matching_row['X18'].values[0]
x20_value = matching_row['X20'].values[0]
x21_value = matching_row['X21'].values[0]
x22_value = matching_row['X22'].values[0]
x27_value = matching_row['X27'].values[0]

Please enter the value of x1a duration from 6,16,17,18,19,20,2121
Please enter the value of x1b duration from 6,16,17,18,19,20,216
Please enter the value of x1c duration from 6,16,17,18,19,20,216


In [33]:
#Analysing first case
task = [('X1A',{'Duration':x1a_duration}),('X1B',{'Duration':x1b_duration}),('X1C',{'Duration':x1c_duration}),('X2',{'Duration':1}),('X3',{'Duration':1}),
        ('X4',{'Duration':1}),('X5',{'Duration':x5_value}),('X6',{'Duration':x6_value}),('X7',{'Duration':x7_value}),('X8',{'Duration':1}),
        ('X9',{'Duration':2}),('X10',{'Duration':1}),('X11',{'Duration':1}),('X12',{'Duration':1}),('X13',{'Duration':1}),
        ('X14',{'Duration':5}),('X15',{'Duration':7}),('X16',{'Duration':5}),('X17',{'Duration':7}),('X18',{'Duration':x18_value}),
        ('X19',{'Duration':7}),('X20',{'Duration':x20_value}),('X21',{'Duration':x21_value}),('X22',{'Duration':x22_value}),('X23',{'Duration':6}),
        ('X24',{'Duration':8}),('X25',{'Duration':2}),('X26',{'Duration':4}),('X27',{'Duration':x27_value}),('X28',{'Duration':7}),
        ]

# Define dependency
depend = [('X1A','X2'),('X1A','X5'),('X1B','X3'),('X1B','X6'),('X1C','X4'),('X1C','X7'),
          ('X2','X8'),('X5','X8'),('X3','X9'),('X6','X9'),('X4','X10'),('X7','X10'),('X8','X11'),('X8','X14'),('X9','X12'),
          ('X9','X15'),('X10','X13'),('X10','X16'),('X11','X17'),('X14','X17'),('X12','X18'),('X15','X18'),('X13','X19'),
          ('X16','X19'),('X17','X20'),('X18','X21'),('X19','X22'),('X17','X23'),('X18','X24'),('X23','X25'),('X24','X25'),
          ('X25','X26'),('X25','X27'),('X20','X28'),('X26','X28'),('X27','X28'),('X21','X28'),('X22','X28')]

In [34]:
# Create task dictionary
taskDict = {t[0]: t[1]['Duration'] for t in task}

# Create predecessor and successor dictionaries
predecessors = {t[0]: [] for t in task}
successors = {t[0]: [] for t in task}

# Populate predecessor and successor dictionaries
for dep in depend:
    predecessors[dep[1]].append(dep[0])
    successors[dep[0]].append(dep[1])

# Convert the data into a DataFrame for easy viewing
data = {
    'Task': list(taskDict.keys()),
    'Duration': list(taskDict.values()),
    'Predecessors': [', '.join(predecessors[task]) if predecessors[task] else 'None' for task in taskDict],
    'Successors': [', '.join(successors[task]) if successors[task] else 'None' for task in taskDict]
}

df = pd.DataFrame(data)

In [35]:
# Process the data
df = forward_pass(df)
df = backward_pass(df)
df = calculate_slack_and_critical(df)

# Display the DataFrame
df

,Task,Duration,Predecessors,Successors,ES,EF,LS,LF,Slack,Critical
0,X1A,21,None,"X2, X5",0,21,0,21,0,True
1,X1B,6,None,"X3, X6",0,6,11,17,11,False
2,X1C,6,None,"X4, X7",0,6,24,30,24,False
3,X2,1,X1A,X8,21,22,22,23,1,False
4,X3,1,X1B,X9,6,7,19,20,13,False
5,X4,1,X1C,X10,6,7,31,32,25,False
6,X5,2,X1A,X8,21,23,21,23,0,True
7,X6,3,X1B,X9,6,9,17,20,11,False
8,X7,2,X1C,X10,6,8,30,32,24,False
9,X8,1,"X2, X5","X11, X14",23,24,23,24,0,True


In [36]:
G = nx.DiGraph()
     
#add nodes and linkegs
G.add_nodes_from(task)
G.add_edges_from(depend)

In [37]:
# initialize the critical path package
proj = Node('Project')

#add task and duration

for t in task:
  proj.add(Node(t[0],duration=t[1]["Duration"] ))

#add dependency

for d in depend:
  proj.link(d[0],d[1])

# update

proj.update_all()

# proj.get_critical_path() will return a list of nodes
# however, we want to store them as strings so that they can be easily used for visualization later

crit_path = [str(n) for n in proj.get_critical_path()]

# get the project completion time.

total_time = proj.duration

print(f"The critical path: {crit_path}")
print("-"*45)
print(f"The project completion time: {total_time} weeks")

The critical path: ['X1A', 'X5', 'X8', 'X14', 'X17', 'X23', 'X25', 'X26', 'X28']
---------------------------------------------
The project completion time: 55 weeks


In [38]:
# Define critical edges
crit_edges = [(n, crit_path[i+1]) for i, n in enumerate(crit_path[:-1])]

# Get edge data for Plotly
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos_nodes[edge[0]]
    x1, y1 = pos_nodes[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

# Critical edges (red line)
crit_edge_x = []
crit_edge_y = []
for edge in crit_edges:
    x0, y0 = pos_nodes[edge[0]]
    x1, y1 = pos_nodes[edge[1]]
    crit_edge_x.append(x0)
    crit_edge_x.append(x1)
    crit_edge_x.append(None)
    crit_edge_y.append(y0)
    crit_edge_y.append(y1)
    crit_edge_y.append(None)

# Trace for regular edges
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=2, color='#888'),
    hoverinfo='none',
    mode='lines')

# Trace for critical edges
crit_edge_trace = go.Scatter(
    x=crit_edge_x, y=crit_edge_y,
    line=dict(width=5, color='red'),
    hoverinfo='none',
    mode='lines')

# Node trace
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos_nodes[node]
    node_x.append(x)
    node_y.append(y)

# Set hover text with EST, EFT, LST, LFT, Slack from pandas DataFrame
node_text = []
for node in G.nodes():
    est = df[df['Task'] == node]['ES'].values[0]
    eft = df[df['Task'] == node]['EF'].values[0]
    lst = df[df['Task'] == node]['LS'].values[0]
    lft = df[df['Task'] == node]['LF'].values[0]
    slack = df[df['Task'] == node]['Slack'].values[0]

    node_text.append(f"Task: {node}<br>"
                     f"EST: {est}<br>"
                     f"EFT: {eft}<br>"
                     f"LST: {lst}<br>"
                     f"LFT: {lft}<br>"
                     f"Slack: {slack}")

# Node trace for Plotly with hover text
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    textposition='bottom center',
    hoverinfo='text',
    text=[node for node in G.nodes()],
    hovertext=node_text,
)

# Create figure with regular and critical edges
fig = go.Figure(data=[edge_trace, crit_edge_trace, node_trace],
                layout=go.Layout(
                    title='<br>Interactive Critical Path Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=0),
                    annotations=[dict(
                        text="Critical Path Network",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002
                    )],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False))
                )

fig.show()
